In [3]:
using MAT, Statistics, LinearAlgebra, Random, Plots, NLsolve, ForwardDiff

ArgumentError: ArgumentError: Package NLsolve not found in current path.
- Run `import Pkg; Pkg.add("NLsolve")` to install the NLsolve package.

Function that reads in all the variables optained that are used in line 93 and onwards and are defined before that.

In [2]:
include("src/mvnrnd.jl")

mvnrnd (generic function with 1 method)

Function that reads in all the variables optained that are used in line 93 and onwards and are defined before that and turns them into julia variables.

In [3]:
function string_as_varname(s::AbstractString,v::Any)
    s=Symbol(s)
    if size(v,2) == 1 # check if v is a column vector
        @eval (($s) = vec($v))
    else
        @eval (($s) = ($v))
    end
end

file = matopen("simulationData.mat")



varnames = names(file) 
#[IO, price, Omega, data, labor, mu, aggtfp, alpha, grossy, stfp, beta, agggdp, Sigma, capital, vadd]
for varname in varnames
    val = read(file, varname)
    # Instantiate the variable with its name in the current scope
    string_as_varname(varname,val)
    end

close(file)

ErrorException: File "simulationData.mat" does not exist and create was not specified

We now want to figure out what every variable stands for, and where it comes from.
So far we know:
- **IO**: Input-Output Matrix
- **STFP** Sectoral TFP Growth per anno
- **$\Omega$**: I think this is the same as in *Definition 3* Es gilt $$(\Omega)_i^j = \frac {(\mathit{IO})_i^j} {\sum_{k=0}^N(\mathit{IO}_k^j)}$$ Also der Anteil eines Sektors $i$ an der Gesamptproduktion des Sektors $j$
- **$\Sigma$**: Covarianz von SFTP
- **$\mu$**: Rowwise mean of stfp. So the mean annual groth of each sector
- **data**: quantity data matrix
- **price**: price data matrix. I think price and data are for our anyalsis quite useless, they are used as building blocks
- **grossy** nominal gross output, was produziert sektor i
- **vadd** nominal value added - Anteil der Einkommen die die Faktoren erzielen.
- **labor** nominal labor
- **capital** nominal captial
- **alpha**: vadd / grossy. Factor (Arbeit, etc.) share by industry. (Alles was nicht von anderen Sektoren bezogen wird).
- **beta**: `beta = (I-diagm(1-alpha)*Omega)' * grossy(:,year-1959)` this is then normalized and filtered. Anteil der Sektoren an der Endnachfrage.

$\lambda$ = total sales / sum(final sales)
(inv(I - diagm(1 .- alpha) * Omega) = Leontief Invers um von final sales auf total sales zu kommen

In [4]:
#domar_weights = (beta' * inv(I - diagm(1 .-alpha)* Omega))';
domar_weights = λ = (inv(I - diagm(1 .- alpha) * Omega)' * beta);

MethodError: MethodError: no method matching -(::Int64, ::typeof(alpha))

Closest candidates are:
  -(::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8})
   @ Base int.jl:85
  -(::T, !Matched::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}
   @ Base int.jl:86
  -(::Union{Int16, Int32, Int64, Int8}, !Matched::BigInt)
   @ Base gmp.jl:547
  ...


In [5]:
cum_stfp = cumsum(log.(1 .+ stfp)',dims=1);
cum_stfp_4year = cum_stfp[:,begin:4:end];
covariance_4year = Statistics.cov(cum_stfp_4year,dims=2,corrected=false);

UndefVarError: UndefVarError: `stfp` not defined

Some elasticities, explaination can be found in section 6

In [6]:
epsilon = ε = .5; #substitution of intermediates with work
theta = θ = 0.001; #substitution elasticity for production factors 
sigma = σ =.9; #substitution elasticity for consumers

Again, not used anymore

In [7]:
trials = 1; #number of draws
#clear Shocks LShocks;
GDP = zeros(trials,1);
variances = (movingvar(stfp',5)'); #rolling estimate of variance of TFP
var_cri = variances[:,22];

var_cri = diag(Sigma)*2; #crisis episode variances



UndefVarError: UndefVarError: `stfp` not defined

L is the steady state allocation of labor. (Wie die Arbeit auf die Sektoren aufgeteilt ist)
$$L_i = (\beta^T (I - \Omega + \text{diag}(\alpha)\Omega)^{-1})^T \cdot \alpha = (I - (I - \text{diagm}(\alpha)) * \Omega)^{-1}\beta) \cdot \alpha = \lambda \cdot \alpha$$


In [9]:
#L = (beta'*inv(I-diagm(1 .- alpha)*Omega))'.*alpha; #steady-state allocation of labor
L = domar_weights .* alpha;

Because later down we use this all a few times I am now going to write a function that extracts $\beta,\alpha,\lambda$ and $L$ for a given year

In [1]:
function getVariables(year)
    IO = data[data[:,1] .== year,:]
    temp = [8,60,62,80:88...]
    IO = IO[setdiff(1:end,temp),setdiff(1:end,[1,2,3,4,5,94])]
    IO = IO[:,setdiff(1:end,temp)]
    Ω = IO ./ sum(IO,dims=2)
    α = vadd[:,year-1959] ./ grossy[:,year-1959];
    β = (grossy[:,year-1959]' * (I-diagm(1 .- α)*Ω))';
    β[beta .< 0] .= 0
    β = β / sum(β)
    λ = (inv(I - diagm(1 .- alpha)*Ω)' * β);
    L = λ .* α
    
    return α,β,Ω,L,λ

end

getVariables (generic function with 1 method)

Here comes the randomness. As far as i can tell the generated vector $x$ has mean $\mu = -\frac 1 2 \text{Var}(\mathit{stfp})$ and $\sigma = \text{Var}(\mathit{stfp})$ Because $\sigma$ is a Diagonal Matrix, there is no correlation between each random number, what makes this rather perculiar. Intuition wise I would say, that this implies, that the shocks are independent of one-another. (As described in  Chapter 6)

$$A_i = e^{x_i}$$

The `fmincon` function is abused in my opinion. The problem as stated is: $$\min_x 0 \quad \text{s.t} \,\, c(x) = 0$$
However this is simply: Find $x$ so that $$c(x) = 0$$And for this the `fsolve` function can be utilized.

Lets analyze what $c(x)$ actually is now. (Only `Out` is of importance, the other return values are either unused arguments in the `fmincon` function or the derivatives of `Out`, to make the solver more efficient.



In [2]:
function Simulation_Derivs(X, A, beta, Omega, alpha, epsilon, theta, sigma,L) # no reallocation of labor
    N = length(alpha);
    p = X[1:N];
    y = X[N+1:2*N];

    q = (Omega* (p .^ (1-theta))) .^ (1/(1-theta));
    w = p .* (A .^ ((epsilon-1)/epsilon)) .* (alpha.^(1/epsilon)) .* (y.^(1/epsilon)) .* (1 ./ L) .^ (1/epsilon);
    C = w'*L;

    
    Out = zeros(2*N)
    Out[1:N] = p - (diagm(A) ^ (epsilon-1) * (alpha.* w .^ (1-epsilon) + (1 .- alpha) .* q .^ (1-epsilon))).^(1/(1-epsilon));
    Out[N+1:2*N] = y' - y' * diagm(p) ^ epsilon * diagm(A)^(epsilon-1) * diagm(q)^(theta-epsilon) * diagm(1 .- alpha) * Omega * diagm(p)^(-theta) - beta' * diagm(p)^(-sigma) * C;
    
    return Out


    
    outineq = [];
    outineq2 = [];
    
    
    DQDP = bsxfun(@times, (q.^theta), (p.^(-theta))').* Omega; % 
    DWDP = diag((A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon)).*(1./L).^(1/epsilon)); %checked
    DWDY = (1/epsilon)*diag(p.*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon-1)).*(L).^(-1/epsilon)); %checked
    DCDP = DWDP'*L; %checked
    DCDY = DWDY'*L;%checked
    DOut1DP = eye(N) - diag(diag(A)^(-1)*((alpha.*(w.^(1-epsilon))+(1-alpha).*(q.^(1-epsilon)))).^(epsilon/(1-epsilon)))*...
        (diag(alpha)*diag(w.^(-epsilon))*DWDP+diag(1-alpha)*diag(q.^(-epsilon))*DQDP);

    DOut1DY =  -diag(diag(A.^(-1))*((alpha.*(w.^(1-epsilon))+(1-alpha).*(q.^(1-epsilon)))).^(epsilon/(1-epsilon)))*...
        (diag(alpha)*diag(w.^(-epsilon))*DWDY);

    DOut2DP = -(epsilon * diag(p.^(-theta))*Omega'*diag((p.^(epsilon-1)).*(y).*(q.^(theta-epsilon)).*(1-alpha).*(A.^(epsilon-1)))...
        +(theta-epsilon)*diag(p.^(-theta))*Omega'*diag((p.^(epsilon)).*(y).*(q.^(theta-epsilon-1)).*(1-alpha).*(A.^(epsilon-1)))*DQDP ...
        -sigma*diag(beta.*p.^(-sigma-1))*C+ bsxfun(@times, beta.*(p.^(-sigma)), DCDP')...
        - theta* diag(p.^(-theta-1)).*diag(Omega'*diag((p.^(epsilon)).*(q.^(theta-epsilon)).*(1-alpha).*(A.^(epsilon-1)))*y));

    DOut2DY = eye(N) - (diag(p)^epsilon*diag(A)^(epsilon-1)*diag(q)^(theta-epsilon)*diag(1-alpha)*Omega*diag(p)^(-theta))' - bsxfun(@times, beta.*(p.^(-sigma)), DCDY');
    OutDeriv = [DOut1DP DOut1DY; DOut2DP DOut2DY]';
    

end

ErrorException: syntax: invalid syntax "1./"; add space(s) to clarify

Für die Elemente von $\Omega$ verwenden wir $\omega$. Potenzieren wird im folgenden Komponentenweise verstanden. W steht für wages. $\omega$ im Paper ist vielleicht wie ein $\alpha$ zu verstehen

$q$ ist der Preis der Intermediärgüter, $w$ der Preis der Arbeit (Lohn)

\begin{align}
q_i &= (\sum_{j=1}^{76} \omega_{i,j}p_j^{1-\theta}) ^ {\frac 1 {1 - \theta}} \\
\mathcal w_i &= p_i (A_i ^ {\frac{\varepsilon -1}{\epsilon}} \alpha_i ^ {\frac 1 \varepsilon} y_i ^ {\frac 1 \varepsilon} L_i ^ {\frac {-1}{\varepsilon}}) = p_i (A_i^{\varepsilon-1}\alpha_i y_i L_i^{-1})^{1/\varepsilon} \\
C &= \langle \mathcal w, L \rangle = \text{GDP}
\end{align}

Wir wollen nun Lösen: Finde $p, y \in \mathbb R^{76}$ sodass $\forall 1 \leq i \leq 76$ gilt:

\begin{align}
p_i &= (A_i^{\varepsilon -1} (\alpha_i \mathcal w_i^{1 - \varepsilon} + (1 - \alpha_i) q_i ^ {1- \varepsilon})) ^ {1/(1-\varepsilon)} \\
y_i &= p_i^{-\theta} \cdot \sum_{j=1}^{76} (\omega_{j,i} p_j^\varepsilon A_j^{\varepsilon -1} q_j^{\theta - \varepsilon} (1 - \alpha_j) y_j) - C p_i^{-\sigma} \beta_i
\end{align}

oder umgeschrieben 

\begin{align}
    A_i p_i &=  ((\alpha_i \mathcal w_i^{1 - \varepsilon} + (1 - \alpha_i) q_i ^ {1- \varepsilon})) ^ {1/(1-\varepsilon))} \\
    y_i  &= p_i^{-\theta} \cdot \sum_{j=1}^{76}  (A_j^{\varepsilon -1} \omega_{j,i} p_j^\varepsilon q_j^{\theta - \varepsilon} (1 - \alpha_j) y_j) - p_i^{-\sigma} C \beta_i =  p_i^{-\theta} \cdot \sum_{j=1}^{76}  (A_j^{\varepsilon -1}  p_j^\varepsilon (1 - \alpha_j) \frac{q_j^{\theta}}{q_j^\varepsilon}  \omega_{j,i }y_j) - p_i^{-\sigma} C \beta_i  

\end{align}


In [41]:
function problem(X, A, β, Ω, α, ε, θ, σ,L)
    
    N = length(α)
    p = @view X[1:N]
    y = @view X[N+1:end]
    
    Out = @MVector zeros(eltype(X),2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    Out[1:N] = p - (A .^ (ε - 1) .* (α .* w .^ (1- ε) + (1 .- α) .* q .^ (1 - ε))) .^ (1/(1-ε))
    Out[N+1:end] = y - p .^ (-θ) .* (Ω'  * (p .^ ε .* A .^ (ε-1) .* q .^(θ-ε) .* (1 .- α) .* y)) - C * p .^(-σ) .* β
    
    return Out
end

problem (generic function with 1 method)

In [25]:
function Jacobian(X, A, β, Ω, α, ε, θ, σ,L)
    N = length(α)
    p = X[1:N]
    y = X[N+1:end]
    
    Out = zeros(2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    
    DQDP = ((q.^θ) .* (p.^(-θ))') .* Ω
    DWDP = diagm((A .^ ((ε-1)/ε)) .* (alpha .^ (1/ε)).*(y .^ (1/ε)) .* L .^ (-1/ε));
    DWDY = (1/ε) *diagm(p .* (A .^ ((ε-1)/ε)) .* (α.^(1/ε)).*(y.^(1/ε-1)).*(L).^(-1/ε));
    DCDP = DWDP'*L;
    DCDY = DWDY'*L;

    DOut1DP = I - diagm(diagm(A)^(-1) * ((.*(w.^(1-ε))+(1 .- α).*(q.^(1-ε)))).^(ε/(1-ε))) * (diagm(α)*diagm(w .^(-epsilon))*DWDP + diagm(1 .- α)*diagm(q.^(-ε))*DQDP);
    DOut1DY =  -diagm(diagm(A.^(-1))*((α.*(w.^(1-ε))+(1 .- α).*(q.^(1-ε)))).^(ε/(1-ε)))* (diagm(α)*diagm(w.^(-ε))*DWDY);

    DOut2DP = -(ε * diagm(p.^(-θ))*Ω'*diagm((p.^(ε-1)).*(y).*(q.^(θ-ε)).*(1 .- α).*(A.^(ε-1))) 
        + (θ-ε)*diagm(p.^(-θ))*Ω'*diagm((p.^(ε)).*(y).*(q.^(θ-ε-1)).*(1 .- α)
            .*(A.^(ε-1)))*DQDP - σ*diagm(β.*p.^(-σ-1))*C + β.*(p.^(-sigma)) .* DCDP' 
        - θ* diagm(p.^(-θ-1)).*diagm(Ω'*diagm((p.^(ε)).*(q.^(θ-ε)).*(1 .- α).*(A.^(ε-1)))*y));

    DOut2DY = I - (diagm(p)^ε * diagm(A)^(ε-1)*diagm(q)^(θ-ε)*diagm(1 .- α)*Ω*diagm(p)^(-θ))' -beta.*(p.^(-sigma)).* DCDY';
    

    [DOut1DP DOut1DY; DOut2DP DOut2DY]'

end

Jacobian (generic function with 1 method)

In [26]:
function f!(F,x)
   F[1:152] = problem(x, A, beta, Omega, alpha, epsilon, theta, sigma,L);
end

f! (generic function with 1 method)

In [27]:
function j!(J,x)
    J[1:152,1:152] .= Jacobian(x, A, beta, Omega, alpha, epsilon, theta, sigma,L);
end

j! (generic function with 1 method)

In [28]:
A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));
#x0 = [exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A));(beta' * inv(I- diagm(1 .- alpha)*Omega))'./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))]  #judicious choice of starting values
x0 = [exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A));domar_weights ./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))];

Die Formel für das GDP lautet

$$GDP = \sum_{i=1}^{76} p_i A_i^{\frac {\varepsilon -1} {\varepsilon}} \alpha_i^{{\frac 1 \varepsilon}} y_i^{\frac{1} {\varepsilon}} L_i^{\frac{-1} {\varepsilon}} L_i = \langle w,L \rangle$$

In [38]:
trials =  100
GDP = zeros(trials)
λ_sim = zeros(76,trials)
for i ∈ 1:trials
    A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));
    x0 = [exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A));domar_weights ./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))];
    x = nlsolve(f!,x0,iterations = 5000, xtol = 10^-14,ftol = 10^-14);

    if converged(x)
        p = x.zero[1:76] 
        y = x.zero[77:152]
        GDP[i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L;
        λ_sim[:,i] .= (p .* y) / (GDP[i]) 
    else
        println("No converence")
    end
end

No converence


No converence


No converence


No converence


No converence


InterruptException: InterruptException:

In [ ]:
for year = 1960:2005

  temp=[8,60,62,80:88];

  IO = data(find(data(:,1)==year),:);
  IO(:,[1 3 4 5 94]) = []; % delete year, gross output, capital, labor, noncompetitive imports
  IO(temp,:) = []; % reove government sectors, and sectors with no gross sales
  Ind = IO(:,1); %store industry names
  IO(:,1) = [];
  IO(:,temp) =[];
  Omega = bsxfun(@rdivide, IO, sum(IO,2));
  %Omega = diag(1-vadd(:,year-1959)./grossy(:,year-1959))*Omega; #scale IO table by intermediate input share
  alpha = (vadd(:,year-1959)./grossy(:,year-1959)); % set the factor share by industry
  N = length(Omega);
  beta = grossy(:,year-1959)'*(eye(N)-diag(1-alpha)*Omega);
  beta(beta<0) = 0; %remove industries with negative implied final sales
  beta = beta/sum(beta); %normalize consumption vector to sum to unity.
  beta = beta';
  lambda(:,count) = (beta'*inv(eye(N)-diag(1-alpha)*Omega))';
  count = count+1;
end

In [30]:
λ = zeros(76,2005-1959)

for year ∈ 1960:2005
    IO = data[data[:,1] .== year,:]
    temp = [8,60,62,80:88...]
    IO = IO[setdiff(1:end,temp),setdiff(1:end,[1,2,3,4,5,94])]
    IO = IO[:,setdiff(1:end,temp)]
    Ω = IO ./ sum(IO,dims=2)
    α = vadd[:,year-1959] ./ grossy[:,year-1959];
    β = (grossy[:,year-1959]' * (I-diagm(1 .- α)*Ω))';
    β[beta .< 0] .= 0
    β = β / sum(β)

    
    λ[:,year - 1959] .= (inv(I - diagm(1 .- alpha)*Ω)' * β);
    
end


In [32]:
mean(λ,dims = 2) * (std(diff(log.(λ_sim),dims=2)))

76×1 Matrix{Float64}:
 0.008107237853817287
 0.0007523135864479131
 0.00031237412691633056
 0.0003789911446114758
 0.0004894700782857832
 0.0008622142459538943
 0.0052101226649808935
 0.018553730503815896
 0.0024250054716087365
 0.0012934304482314723
 0.002217343938996541
 0.0060222163322303695
 0.005501985315388467
 ⋮
 0.0011648769760409742
 0.0008023019981786588
 0.0015585425294140877
 0.003961932922477893
 0.0007967227287495287
 0.003665569047136084
 0.0009059796715378963
 0.0021795041099331425
 0.001737222694811052
 0.003107769586549516
 0.0007895782027387948
 0.0044589069478056026

Here we do this simulation again, but we just shock certain sectors

In [ ]:
M = 10;
GDP = zeros(2*M,1);
grid = linspace(1.0, a, M);
list = [7;53;8]
for k = 1:length(list)
        Ind = list(k);
        grid = linspace(1.0, a, M);
    for j = 1:M
            A = ones(N,1);
            A(Ind) = grid(j);
            [Soln,~,exitfl] = knitromatlab(@(X) trivial(X),init,[],[],[],[],[],[], @(X)Simulation_Derivs(X,  A, beta, Omega, alpha, epsilon, theta, sigma,L),[],[],'Knitro_options.opt');
            if exitfl == 0
              GDP(j,k) = (Soln(1:N).*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(Soln(N+1:2*N).^(1/epsilon)).*(1./L).^(1/epsilon))'*L;
            end
            init = Soln;
    end


    grid = linspace(1.0, b, M);
    init = [ones(N,1);(beta'*inv(eye(N)-diag(1-alpha)*Omega))'];
    for j = 1:M
            A = ones(N,1);
            A(Ind) = grid(j);
                [Soln,~,exitfl] = knitromatlab(@(X) trivial(X),init,[],[],[],[],[],[], @(X)Simulation_Derivs(X,  A, beta, Omega, alpha, epsilon, theta, sigma,L),[],[],'Knitro_options.opt');
                if exitfl == 0
                    GDP(j+M,k) = (Soln(1:N).*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(Soln(N+1:2*N).^(1/epsilon)).*(1./L).^(1/epsilon))'*L;
                end
                init = Soln;
    end
    GDP(1:M,k) = flipud(GDP(1:M,k));
end

In [ ]:
year = 1982
M = 10
A = ones(76,1);
a = 0.7;
b = 1.3;

ε = .3; #Elasticity of substitution between VA and intermediates
θ = 0.0001; #Elasticity of substitution between intermediates
σ = .4; #Elasticity of substitution between in consumption


(α,β,Ω,L,λ) = getVariables(year);

x0 = [ones(76,1);λ];

GDP = zeros(20,3)
sectors = [7 8 53]
sectorShocks = LinRange(1,a,M)
for i in 1:3
    for k in 1:M
        A = ones(76)
        A[sectors[i]] = sectorShocks[k]
        
        function f!(F,x)
            F[1:152] = problem(x, A, β, Ω, α, ε, θ, σ,L);
        end
        
        function j!(J,x)
            J[1:152,1:152] .= Jacobian(x, A, beta, Omega, alpha, epsilon, theta, sigma,L);
        end
        
        x = nlsolve(f!,j!,x0,iterations = 500, xtol = 10^-8,ftol = 10^-8);
        
        if converged(x)
            p = x.zero[1:76] 
            y = x.zero[77:152]
            GDP[k,i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L
            x0 =  x.zero
        else
            print("No convergence")
            p = x.zero[1:76] 
            y = x.zero[77:152]
            GDP[k,i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L
            x0 =  x.zero
        end
    end

end

